In [1]:
import numpy as np
from collections import deque
import heapq
import math

In [2]:
adj_mat = np.zeros((4,4))
np.fill_diagonal(adj_mat, 1)
adj_mat[0, 1] = 1
adj_mat[1, 2] = 2
adj_mat[2, 0] = 3
adj_mat[0, 3] = 4
adj_mat

array([[1., 1., 0., 4.],
       [0., 1., 2., 0.],
       [3., 0., 1., 0.],
       [0., 0., 0., 1.]])

In [3]:
adj_list = {
    'a' : [('b', 1), ('d', 4)],
    'b' : [('c', 2)],
    'c' : [('a', 3)],
    'd' : []
}

def transform_to_matrix (adj_list):
    
    mat = np.full((len(adj_list), len(adj_list)), np.nan)
    np.fill_diagonal(mat, 1)
    
    index = {vertex: i for i, vertex in enumerate(adj_list.keys())}
    for origin, edges in adj_list.items():
        for destination, weight in edges:
            mat[index[origin], index[destination]] = weight
            
    return mat

transform_to_matrix(adj_list)

array([[ 1.,  1., nan,  4.],
       [nan,  1.,  2., nan],
       [ 3., nan,  1., nan],
       [nan, nan, nan,  1.]])

In [4]:
class Vertex():
    def __init__(self, id: str) -> None:
        self._id = id
        self._adjacent = dict()
        # Set distance to infinity for all nodes
        self._distance = math.inf
        # Mark all nodes unvisited        
        self._visited = False  
        # Predecessor
        self._previous = None
        
    def get_id(self):
        return self._id
    
    def set_id(self, id):
        self._id = id
        
    def add_neighbour(self, neighbour, weight=0):
        self._adjacent[neighbour] = weight
   
    def get_adjacent(self):
        return self._adjacent
    
    def get_weight(self, neighbour):
        return self._adjacent[neighbour] 
    
    def set_distance(self, dist):
        self._distance = dist

    def get_distance(self):
        return self._distance

    def set_previous(self, prev):
        self._previous = prev

    def get_previous(self):
        return self._previous

    def set_visited(self):
        self._visited = True

    def is_visited(self):
        return self._visited

    def __str__(self):
        adj_mat = {v.get_id(): w for v, w in self.get_adjacent().items()}       
        return str(self._id) + ' adjacent: ' + str(adj_mat)
    
    def __lt__(self, other):
        return self._distance < other.get_distance()
  
        

class EVWeightedGraph():
    def __init__(self) -> None:
        self._vertices = {}
        
    def __repr__(self) -> str:
        result = ''
        for v in self._vertices.values():
            result += str(v) + '\n'
        return result.strip()

    def add_vertex (self, integer_vertex: int) -> Vertex:
        '''
        Add a vertex to the graph where the argument integer_vertex
        is guaranteed to be unique and serves to identify the vertex
        and also represents the vertex weight.
        '''
        v = Vertex(integer_vertex)
        self._vertices[integer_vertex] = v
        return v
    
    def get_vertex(self, id):
        if id in self._vertices:
            return self._vertices[id]
        return

    def add_edge (self, v1: int, v2: int, weight):
        '''
        Adds a weighted edge between two vertices identified by their
        unique integer vertex id/weight. 
        '''   
        if v1 not in self._vertices:
            self.add_vertex(v1)
        if v2 not in self._vertices:
            self.add_vertex(v2)
            
        self._vertices[v1].add_neighbour(self._vertices[v2], weight)
        self._vertices[v2].add_neighbour(self._vertices[v1], weight)
        
        
    def dijkstra_spf (self, start):
    
    # Set the distance for the start node to zero
        start.set_distance(0)

        # Initialize priority queue with the start vertex
        unvisited_queue = []
        heapq.heappush(unvisited_queue, start)

        while unvisited_queue:
            # Pops a vertex with the smallest distance
            current = heapq.heappop(unvisited_queue)
            current.set_visited()

            # Visit all adjacent vertices
            for next in current.get_adjacent():
                if next.is_visited():
                    continue

                new_dist = current.get_distance() + current.get_weight(next)
                if new_dist < next.get_distance():
                    next.set_distance(new_dist)
                    next.set_previous(current)
                    # Push the updated vertex back into the priority queue
                    heapq.heappush(unvisited_queue, next)

        
    def list_shortest_path_by_weight (self, start: int, destination: int):
        '''
        Returns a list of alternative vertex and edge weights as a 
        list of integers of the form:

          [start, weight, ... weight, destination]

        For example, in a graph with two vertices '0' and '1' with a
        single edge '42', the function returns [0, 42, 1].

        If no path exists between start and destination, an empty 
        list is returned.  
        '''
        vs, vd = self.get_vertex(start), self.get_vertex(destination)
        
        if not vs or not vd:
            return []
        
        self.dijkstra_spf(vs)
        
        path = []
        current = vd
        while current:
            path.append(current.get_id())
            prev = current.get_previous()
            if prev is not None:
                path.append(current.get_weight(prev))
            current = prev
        path.reverse()
        
        if path[0] != start:
            return []
    
        return path
    
evwg = EVWeightedGraph()

for v in range(5):
    evwg.add_vertex (v)

evwg.add_edge (0, 1, 3)
evwg.add_edge (1, 2, 1)
evwg.add_edge (0, 4, 8)
evwg.add_edge (0, 3, 7)
evwg.add_edge (4, 3, 3)
evwg.add_edge (1, 3, 4)
evwg.add_edge (3, 2, 2)
evwg

0 adjacent: {1: 3, 4: 8, 3: 7}
1 adjacent: {0: 3, 2: 1, 3: 4}
2 adjacent: {1: 1, 3: 2}
3 adjacent: {0: 7, 4: 3, 1: 4, 2: 2}
4 adjacent: {0: 8, 3: 3}

In [5]:
# Note that the shortest path ignores the edge (1, 3) to use a shorter
# edge weight route via vertices 1, 2, 3
assert evwg.list_shortest_path_by_weight (1, 3) == [1, 1, 2, 2, 3]


In [26]:
spectrum_colours = [
    'red',
    'orange',
    'yellow',
    'green',
    'cyan',
    'blue',
    'violet'
]

fave_colours = [
    ('Rui', 'orange'),
    ('Pavithra', 'yellow'),
    ('Yang', 'blue'),
    ('Cameron', 'red'),
    ('Liangwei', 'violet'),
    ('Roland', 'green'),
    ('Ali', 'cyan')
]


class TreeVertex:

    def __init__ (self, _value = None):
        # User domain payload of the TreeVertex
        self._value = _value
        
        # Left and right sided children
        self._left = None
        self._right = None
        index = {v:i for i, v in enumerate(spectrum_colours)}
        self._order = index.get(_value[1], -1)
        
    def get_value (self):
        return self._value
    
    def set_value (self, _value):
        self._value = _value
        
    def get_left (self):
        return self._left

    def set_left (self, new_left):
        self._left = new_left
        
    def get_right (self):
        return self._right
    
    def set_right (self, new_right):
        self._right = new_right
        
        
    def get_order(self):
        return self._order
        
    def __str__(self) -> str:
        return str(self._value)
    
    def insert_value(self, value):
        v = TreeVertex(value)
        if v.get_order() < self._order:
            if self.get_left() is None:
                self.set_left(v)
            else:
                self.get_left().insert_value(value)
       
        if v.get_order() > self._order:
            if self.get_right() is None:
                self.set_right(v)
            else:
                self.get_right().insert_value(value)        
            
    def inorder(self):
        result = []
        if self.get_left():
            result.extend(self.get_left().inorder())
        result.append(self.get_value())
        if self.get_right():
            result.extend(self.get_right().inorder())
            
        return result

    def preorder(self):
        result = [self.get_value()]
        if self.get_left():
            result.extend(self.get_left().preorder())
        if self.get_right():
            result.extend(self.get_right().preorder())

        return result
    
    def postorder(self):
        result = []
        if self.get_left():
            result.extend(self.get_left().postorder())
        if self.get_right():
            result.extend(self.get_right().postorder())
        result.append(self.get_value())
        
        return result

    
class ColourTree ():
    def __init__(self):
        self._root = None  
        
    def __str__(self) -> str:
        result = ''
        for vertex in self._root.inorder():
            result += str(vertex) + '\n'
        return result.strip()
    
    def insert_value (self, value):
        '''
        Insert a (name, colour) tuple in the BST with ordering 
        determined by the position of the colour in the list 
        "spectrum_colours" such that the in-order traversal of a seven
        element tree where each vertex has a unique colour would read:

         (<name1>, 'orange'), (<name2>, 'yellow') ... (<name7>, 'violet')
        
        '''
        
        if not self._root:
            self._root = TreeVertex(value)
        else:
            self._root.insert_value(value)
        

    def in_order_names(self):
        '''
        Return a list of names for an in-order traversal of the BST
        (excluding the colour). For example, after inserting values
        outlined above, in_order_names() would return the list:

            ['name1', 'name2', ... 'name7']

        If the root of the list is not yet set, returns an empty list.     
        '''
        return [fav[0] for fav in self._root.inorder()]
    
tree = ColourTree()

for colour in fave_colours:
    tree.insert_value(colour)
    
tree.in_order_names()


['Cameron', 'Rui', 'Pavithra', 'Roland', 'Ali', 'Yang', 'Liangwei']

In [27]:
tree = ColourTree()

for colour in fave_colours:
    tree.insert_value(colour)

assert tree.in_order_names() == ['Cameron', 'Rui', 'Pavithra', 'Roland', 'Ali', 'Yang', 'Liangwei']